# Preprocessing the taxi data - Intentionally Blank

**NOTE: This notebook does not need to be executed. A copy of the preprocessed dataframe is saved as a [parquet file](https://parquet.apache.org/)**. For the preparation notebook click [here](./prep.ipynb)

Before preparing and cleaning the taxi dataset, we should first preprocess the CSV to make it smaller. We collected the data from the Chicago Data Portal. We filtered the original dataset by the trip_start_timestamp directly via the API to minimize the initial filesize. To get all the trips from 2016 we used the following query: https://data.cityofchicago.org/resource/wrvz-psew.csv?$where=trip_start_timestamp%20between%20%272016-01-01T00:00:00%27%20and%20%20%272016-12-31T23:59:59%27&$limit=1000000000.
<br>For further information about the dataset and the API click the following link: [Chicago Data Portal - Taxi Trips](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew).

To run this notebook you need to download the dataset from the aforementioned link. And copy it to the "data" folder as "original_taxi_data.csv". Furthermore atleast 16GB of ram is needed to read the CSV and 32GB is recommended because the initial CSV is ~14GB big.

**Dependencies:**
- Pandas
- Pyarrow (conda install pyarrow)
  - Needed for saving to a parquet file

In [1]:
# Importing the libraries
import pandas as pd
import os  
os.makedirs('./data', exist_ok=True) 

In [2]:
# Reading the csv file
# Note: This file is not included in the repository due to its size. Please use the link above to download the file. Loading the file may take a few minutes.
taxi_df = pd.read_csv('data/original_taxi_data.csv')

In [3]:
# Checking for the right time range
taxi_df["trip_start_timestamp"].min(), taxi_df["trip_start_timestamp"].max()

('2016-01-01T00:00:00.000', '2016-12-31T23:45:00.000')

In [4]:
# Checking memory usage for later comparison
taxi_df.memory_usage(deep=True)

Index                                128
trip_id                       3080655883
taxi_id                       5875028507
trip_start_timestamp          2540747120
trip_end_timestamp            2540631248
trip_seconds                   254074712
trip_miles                     254074712
pickup_census_tract            254074712
dropoff_census_tract           254074712
pickup_community_area          254074712
dropoff_community_area         254074712
fare                           254074712
tips                           254074712
tolls                          254074712
extras                         254074712
trip_total                     254074712
payment_type                  2034421040
company                       2256911991
pickup_centroid_latitude       254074712
pickup_centroid_longitude      254074712
pickup_centroid_location      2692429277
dropoff_centroid_latitude      254074712
dropoff_centroid_longitude     254074712
dropoff_centroid_location     2681476484
dtype: int64

In [5]:
taxi_df.head(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,223789629c9e0a01fbab0d787d2664ccdb8355c0,507b1e4d1f39a8a26e7249e6a627f5a0c798dfdafa7b16...,2016-12-31T23:45:00.000,2016-12-31T23:45:00.000,180.0,0.7,NaN,NaN,NaN,NaN,...,1.0,5.75,Cash,City Service,NaN,NaN,NaN,NaN,NaN,NaN
1,a1d390b16ede0f133408103b79dcb56bbd74365e,73b2f5adecea91eeef3900303a07f1b0519a594cffb6b0...,2016-12-31T23:45:00.000,2017-01-01T00:15:00.000,2160.0,5.4,NaN,NaN,NaN,NaN,...,0.0,23.50,Cash,Chicago Taxicab,NaN,NaN,NaN,NaN,NaN,NaN
2,2fffdf0e5b45125ed3fd7027b92e31bd7e7085ef,d41ab2be597b82c3e6b0b0ecccf98883a84db0d9aed4f6...,2016-12-31T23:45:00.000,2017-01-01T00:00:00.000,1080.0,5.1,NaN,NaN,NaN,NaN,...,0.0,15.75,Cash,City Service,NaN,NaN,NaN,NaN,NaN,NaN
3,3c1d5e90e522f7be0bf92c96f5164360d8d02f94,24515782c70f09819506a7724a57e77c78fea60c4dc91d...,2016-12-31T23:45:00.000,2017-01-01T00:00:00.000,780.0,2.9,NaN,NaN,NaN,NaN,...,0.0,11.00,Cash,Sun Taxi,NaN,NaN,NaN,NaN,NaN,NaN
4,d9046368ad0f1ba4cc27c659e9467cd3602bd458,f1eda6f0cb8e48e7fdb5f623a4a5113a84c159fbf73638...,2016-12-31T23:45:00.000,2016-12-31T23:45:00.000,0.0,0.0,NaN,NaN,NaN,NaN,...,0.0,5.00,Credit Card,Suburban Dispatch LLC,NaN,NaN,NaN,NaN,NaN,NaN


We delete irrelevant columns to save as much memory as possible.

In [7]:
#Drop columns pickup_centroid_location, dropoff_centroid_location, fare, tips, tolls, extras, payment_type, pickup_community_area, dropoff_community_area, company
taxi_df = taxi_df.drop(columns=['pickup_centroid_latitude', 'pickup_centroid_longitude', 'dropoff_centroid_latitude', 'dropoff_centroid_longitude', 'fare', 'tips', 'tolls', 'extras', 'payment_type', 'pickup_community_area', 'dropoff_community_area', 'company', 'taxi_id'])

Deleting rows with null values and duplicates is done in this notebook instead of the preparation notebook to ensure that most computers with low memory can run the preparation notebook.

In [8]:
display(taxi_df[taxi_df.isnull().any(axis = 1)].head(5))

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,trip_total,pickup_centroid_location,dropoff_centroid_location
0,223789629c9e0a01fbab0d787d2664ccdb8355c0,2016-12-31T23:45:00.000,2016-12-31T23:45:00.000,180.0,0.7,NaN,NaN,5.75,NaN,NaN
1,a1d390b16ede0f133408103b79dcb56bbd74365e,2016-12-31T23:45:00.000,2017-01-01T00:15:00.000,2160.0,5.4,NaN,NaN,23.50,NaN,NaN
2,2fffdf0e5b45125ed3fd7027b92e31bd7e7085ef,2016-12-31T23:45:00.000,2017-01-01T00:00:00.000,1080.0,5.1,NaN,NaN,15.75,NaN,NaN
3,3c1d5e90e522f7be0bf92c96f5164360d8d02f94,2016-12-31T23:45:00.000,2017-01-01T00:00:00.000,780.0,2.9,NaN,NaN,11.00,NaN,NaN
4,d9046368ad0f1ba4cc27c659e9467cd3602bd458,2016-12-31T23:45:00.000,2016-12-31T23:45:00.000,0.0,0.0,NaN,NaN,5.00,NaN,NaN


In [9]:
# Drop rows with missing values
taxi_df = taxi_df.dropna(how='any',axis=0)
print(f"Number of rows after deleting rows with null values: {len(taxi_df)} ")

Number of rows after deleting rows with null values: 20356209 


In [10]:
display(taxi_df[taxi_df.duplicated(subset=['trip_start_timestamp', 'trip_end_timestamp', 'trip_id'])].head(5))

,trip_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,trip_total,pickup_centroid_location,dropoff_centroid_location


Because no duplicate rows exist in our dataset we skip the deletion part and drop the obsolete trip_id column.

In [11]:
taxi_df = taxi_df.drop(columns=[ "trip_id"])

In [12]:
# Last look at the data
taxi_df.head(5)

,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,trip_total,pickup_centroid_location,dropoff_centroid_location
9,2016-12-31T23:45:00.000,2017-01-01T00:00:00.000,660.0,2.2,1.703108e+10,1.703183e+10,10.25,POINT (-87.6288741572 41.8920726347),POINT (-87.6572331997 41.8852813201)
15,2016-12-31T23:45:00.000,2017-01-01T00:00:00.000,480.0,0.0,1.703108e+10,1.703108e+10,9.00,POINT (-87.6188683546 41.8909220259),POINT (-87.6188683546 41.8909220259)
25,2016-12-31T23:45:00.000,2017-01-01T00:00:00.000,600.0,0.7,1.703108e+10,1.703108e+10,6.50,POINT (-87.6318639497 41.8920421365),POINT (-87.6318639497 41.8920421365)
26,2016-12-31T23:45:00.000,2017-01-01T00:00:00.000,240.0,0.3,1.703108e+10,1.703108e+10,8.00,POINT (-87.6378442095 41.8932163595),POINT (-87.6378442095 41.8932163595)
28,2016-12-31T23:45:00.000,2016-12-31T23:45:00.000,60.0,0.2,1.703108e+10,1.703108e+10,3.75,POINT (-87.6262149064 41.8925077809),POINT (-87.6262149064 41.8925077809)


In [13]:
# Convert trip_seconds to uint32 without losing information
taxi_df = taxi_df.astype({"trip_seconds": "uint32"})

In [14]:
# Optional: If you want to save the preprocessed data as a csv file uncomment the following line
# taxi_df.to_csv('data/taxi_data_preprocessed.csv', index=False)

# Saving the preprocessed data as a parquet file with gzip compression
taxi_df.to_parquet('data/taxi_data_preprocessed.gzip', compression='gzip')